# Convolutional Neural Network

In [2]:
import numpy as np
import torch
import torchvision

In [3]:
transformer = torchvision.transforms.ToTensor()

# Set download to True if the data is not present
mnist_train = torchvision.datasets.FashionMNIST(
                          root="../data",
                          train=True,
                          transform=transformer,
                          download=False)
mnist_test = torchvision.datasets.FashionMNIST(
                          root="../data",
                          train=False,
                          transform=transformer,
                          download=False)

In [4]:
batch_size = 100
train_iter = torch.utils.data.DataLoader(mnist_train,
                                         batch_size,
                                         shuffle=True)
test_iter = torch.utils.data.DataLoader(mnist_test,
                                        batch_size,
                                        shuffle=False)


In [13]:
num_inputs = 28 * 28
num_outputs = 10

class Reshape(torch.nn.Module):
  def forward(self, x):
    return x.view(-1, 1, 28, 28)

model = torch.nn.Sequential(Reshape(),
                            torch.nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
                            torch.nn.ReLU(),
                            torch.nn.MaxPool2d(kernel_size=2, stride=2),
                            torch.nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
                            torch.nn.ReLU(),
                            torch.nn.MaxPool2d(kernel_size=2, stride=2),
                            torch.nn.Flatten(),
                            torch.nn.Linear(7*7*32, num_outputs))

In [14]:
loss = torch.nn.CrossEntropyLoss()

In [15]:
learning_rate = 0.003
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
num_epochs = 6
for epoch in range(num_epochs):
  for X, y in train_iter:
    l = loss(model(X), y)

    optimizer.zero_grad()
    l.backward()
    optimizer.step()

In [17]:
with torch.no_grad():
  correct = 0
  total = 0
  for X, y in test_iter:
    outputs = model(X)
    _, predicted = torch.max(outputs.data, 1)
    total += y.size(0)
    correct += (predicted == y).sum()

  print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 90.19999694824219 %


In [18]:
torch.save(model.state_dict(), '../save/cnn.ckpt')